In [1]:
from pathlib import Path

import numpy as np
import torch
from scipy.stats import entropy
from sklearn.metrics import f1_score, accuracy_score
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader

from ml_utils import CustomDataset
from model.resnet_model import CNNClassifier

In [2]:
device = "mps"

In [3]:
test_dt = CustomDataset(data_path=Path("../datasets/car_dataset"), portion="test")
test_loader = DataLoader(test_dt, batch_size=64, shuffle=True, num_workers=4)

cls = CNNClassifier(num_classes=196, pretrained=True, model_name="resnet18", freeze_layers=0)
cls.load_state_dict(torch.load("../webapp_result/r18_pretrained_freeze0/model.pth", weights_only=True))
cls.eval()
cls = cls.to(device)

In [4]:
from tqdm import tqdm

preds, trues, entropy_list = [], [], []
criterion = CrossEntropyLoss().to(device)

for idx, (x, y) in enumerate(tqdm(test_loader), 1):
    x, y = x.to(device), y.to(device)
    x.requires_grad_(True)

    loss = criterion(cls(x), y)
    cls.zero_grad()
    loss.backward()

    with torch.no_grad():
        adv_logits = cls(x + .007 * torch.sign(x.grad))
        entropy_list.append(entropy(torch.softmax(adv_logits, dim=1).cpu().numpy()))

    predictions = torch.argmax(adv_logits, dim=1)
    preds.append(predictions)
    trues.append(y)

# Concatenate all predictions, true labels, and losses
preds = torch.cat(preds).cpu().numpy()
trues = torch.cat(trues).cpu().numpy()
entropy_list = torch.tensor(np.asarray(entropy_list))

f1_mean = f1_score(trues, preds, average='weighted')
acc_mean = accuracy_score(trues, preds)

100%|██████████| 126/126 [04:10<00:00,  1.99s/it]


In [5]:
print(entropy_list.mean(), entropy_list.std())

tensor(2.6262) tensor(0.8036)


In [6]:
f1_mean

0.12186538816469347

In [7]:
acc_mean

0.11677652033329187